## Тестирование нейросети через REST API для детекции дефектов дорожного полотна
Этот ноутбук предназначен для интерактивного тестирования сервиса детекции дефектов дорожного полотна по REST API. Он позволяет выбрать тестовое изображение, задать порог уверенности и визуально оценить результаты детекции (боксы и классы дефектов).

 ### Запуск ноутбука

1. REST API‑сервер детекции должен быть предварительно запущен. Если он ещё не запущен, то в виртуальном окружении проекта выполните:  
```bash
python main.py
```

2. Запустите первую ячейку ноутбука. Отобразится виджет для выбора и отправки изображения на сервер FAST API для обработки.


### Использование интерфейса
Интерфейс включает:  

- выпадающий список выбора входного изображения (файлы из папки test_images);
- ползунок порога уверенности (confidence_threshold модели);
- кнопку «Детектировать» — отправка выбранного изображения на REST API;
- выходное поле — текстовый лог и визуализация результатов.  

Порядок работы:  
- Выберите изображение и при необходимости настройте порог уверенности;
- Нажмите кнопку «Детектировать». Изображение загрузится из папки test_images, конвертируется в base64 и будет отправлено на сервер. После обработки изображения сервер вернёт, в частности, список обнаруженных дефектов и их рамок, которые накладываются в этом ноутбуке на исходное изображение.

Результат работы:  
- первой фигуре отображается исходное изображение;
- в текстовом поле выводятся:  
    - статус запроса;
    - время обработки;
    - количество детекций;
    - список детекций (класс, уверенность).
- во второй фигуре отображается изображение с нанесёнными прямоугольниками и подписями по результатам детекции.


In [1]:
# Тестирование нейросети через REST API для детекции дефектов дорожного полотна
import requests
import base64
import json
import numpy as np
import cv2
from PIL import Image
import io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output

# Настройка matplotlib для Jupyter
%matplotlib inline

# URL API сервера
API_URL = "http://localhost:8080/api/v1/test/detect"

# Получаем список доступных изображений
test_images_dir = Path("test_images")
image_files = sorted([f for f in test_images_dir.glob("*") 
                      if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']])

# Создаём выпадающий список для выбора изображения
image_dropdown = widgets.Dropdown(
    options=[(f.name, str(f)) for f in image_files],
    description='Изображение:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Слайдер для порога уверенности
confidence_slider = widgets.FloatSlider(
    value=0.2,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Порог уверенности:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Кнопка для детекции
detect_button = widgets.Button(
    description='🔍 Детектировать',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Выходной виджет
output = widgets.Output()


def on_detect_click(b):
    """Детекция дефектов на выбранном изображении"""
    with output:
        clear_output(wait=True)
        
        # Загружаем изображение, если оно еще не загружено или изменилось
        test_image_path = image_dropdown.value
        global img, img_rgb, last_image_path
        
        if 'last_image_path' not in globals() or last_image_path != test_image_path:
            img = cv2.imread(test_image_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            last_image_path = test_image_path
            print(f"✓ Изображение загружено: {img.shape}\n")
        
        # Отображаем исходное изображение 
        fig1, ax1 = plt.subplots(figsize=(12, 8))
        ax1.imshow(img_rgb)
        ax1.set_title(f"Исходное изображение: {Path(test_image_path).name}", fontsize=14)
        ax1.axis('off')
        plt.tight_layout()
        plt.show()
        
        # Конвертация в base64
        success, encoded_image = cv2.imencode('.jpg', cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))
        if not success:
            print("Ошибка кодирования изображения")
            return
        
        image_base64 = base64.b64encode(encoded_image.tobytes()).decode('utf-8')
        
        # Запрос к API
        request_data = {
            "image": image_base64,
            "confidence_threshold": confidence_slider.value
        }
        
        print("Отправка запроса к API...")
        print(f"Порог уверенности: {confidence_slider.value:.2f}\n")
        
        try:
            response = requests.post(
                API_URL,
                json=request_data,
                headers={"Content-Type": "application/json"},
                timeout=10
            )
            response.raise_for_status()
            result = response.json()
            
            print(f"Запрос успешен!")
            print(f"Время обработки: {result['processing_time_ms']:.2f} мс")
            print(f"Обнаружено дефектов: {len(result['detections'])}\n")
            
            if result['detections']:
                print("🔍 Детали детекций:")
                for i, det in enumerate(result['detections'], 1):
                    print(f"   {i}. {det['class_name']} - {det['confidence']:.2%}")
                
                # Визуализация
                fig, ax = plt.subplots(figsize=(12, 8))
                ax.imshow(img_rgb)
                ax.set_title(f"Результаты детекции ({len(result['detections'])} дефектов)", fontsize=14)
                ax.axis('off')
                
                colors = ['red', 'blue', 'green', 'yellow', 'orange', 'purple']
                for i, det in enumerate(result['detections']):
                    bbox = det['bbox']
                    x_min, y_min, x_max, y_max = bbox
                    
                    rect = Rectangle(
                        (x_min, y_min), x_max - x_min, y_max - y_min,
                        linewidth=2, edgecolor=colors[i % len(colors)], facecolor='none'
                    )
                    ax.add_patch(rect)
                    
                    label = f"{det['class_name']} ({det['confidence']:.2%})"
                    ax.text(x_min, y_min - 5, label,
                           color=colors[i % len(colors)], fontsize=10, weight='bold',
                           bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
                
                plt.tight_layout()
                plt.show()
                # График отобразится автоматически через %matplotlib inline
                
                globals()['result'] = result
            else:
                print("Дефекты не обнаружены")
                
        except requests.exceptions.ConnectionError:
            print("Ошибка: Не удалось подключиться к API серверу")
            print("Убедитесь, что сервер запущен: python main.py")
        except Exception as e:
            print(f"Ошибка: {e}")

# Привязываем обработчик
detect_button.on_click(on_detect_click)

# Отображаем интерфейс
display(widgets.VBox([
    widgets.HTML("<h3>ВЫБОР ИЗОБРАЖЕНИЯ И ПАРАМЕТРОВ</h3>"),
    image_dropdown,
    confidence_slider,
    detect_button,
    output
]))


## Классы с датасета  RDD расшифровываются так:  
D00 – продольная трещина (longitudinal crack);  
D10 – поперечная трещина (transverse crack);  
D20 – «крокодилья» трещина / сетка трещин (alligator crack);  
D40 – выбоина (pothole).